## Correlation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import re

In [2]:
biz = pd.read_csv('data/businesses_mtl.csv')
biz.sample(5)

,id,name,url,review_count,categories,rating,latitude,longitude,price,location,phone
419,DAMTCTsSeACXbkSABkhZqQ,Alto,https://www.yelp.com/biz/alto-montr%C3%A9al-2,60,"['sandwiches', 'pizza', 'burgers']",3.5,45.509259,-73.572748,$$,"['3469 Avenue du Parc', 'Montreal, QC H2X 2H6'...",1.514845e+10
382,uyyvYlZRUK3UWMDllDRWxA,Ginkgo,https://www.yelp.com/biz/ginkgo-montr%C3%A9al-2,60,"['bars', 'tapas', 'breakfast_brunch']",4.0,45.512797,-73.561099,$$,"['308 Rue Sainte-Catherine E', 'Montreal, QC H...",1.514379e+10
794,C-wwhNT05pnpPPMWnNWybA,Bistro l'Évidence,https://www.yelp.com/biz/bistro-l-%C3%A9videnc...,37,"['breakfast_brunch', 'brasseries']",4.0,45.519605,-73.572873,$$,"['3817 Rue Saint-Denis', 'Montreal, QC H2W 2M4...",1.514847e+10
498,Y6zqMM_VI16v2icj7NYnsA,Le Boulevardier,https://www.yelp.com/biz/le-boulevardier-montr...,3,"['french', 'cocktailbars']",4.5,45.502440,-73.574400,NaN,"['2050 Rue Mansfield', 'Montreal, QC H3A 1Y9',...",1.514986e+10
44,aLubn-yEUrbL3vlvzK3lVw,Che Churros & Empanadas,https://www.yelp.com/biz/che-churros-and-empan...,19,['argentine'],5.0,45.517670,-73.585220,NaN,"['100 Rue Marie-Anne O', 'Montreal, QC H2W 1B9...",1.514983e+10


In [3]:
df = pd.read_csv('data/yelp_reviews_mtl.csv')
df

,user_url,user_name,user_city,user_rating,user_review,biz_id
0,/user_details?userid=lxGwbMkC_I9D7GhGT23mFQ,Cynthia L.,"Ville-Marie, Montreal, Canada",5,Went here pre-covid with my bf and best friend...,0W4lkclzZThpx3V65bVgig
1,/user_details?userid=IMHvG_ysz6akYIM-GO18kg,Andrea D.,"Le Plateau-Mont-Royal, Montreal, Canada",5,*home delivery review* Amazing. Literally tast...,0W4lkclzZThpx3V65bVgig
2,/user_details?userid=DlNz5SxdBYRaZ_pnyIxn6g,Rhonda B.,"Montreal, Canada",5,I must add a disclaimer to this review since I...,0W4lkclzZThpx3V65bVgig
3,/user_details?userid=gAvuqk1q2uAo2BJzZDLmDQ,Matt M.,"Oklahoma City, OK",5,One of my favorite Quebecois institutions!No t...,0W4lkclzZThpx3V65bVgig
4,/user_details?userid=3VLcNOmumPkwuSoQxTBHTw,Marc I.,"Ajax, Canada",5,Is there anything better that a true Smoked me...,0W4lkclzZThpx3V65bVgig
...,...,...,...,...,...,...
64862,/user_details?userid=XEsPq0Kd7Uxq4Guv0Dj6Wg,Nicole R.,"Montreal, Canada",2,Such a disappointing experience!! Went to Raso...,pJrM4yAZo0Gs63D9UCyCSQ
64863,/user_details?userid=S_sIeCOqy7F4RFs0le1cDQ,Joel S.,"Los Angeles, CA",5,"Delicious food, tasty cocktails, reasonably pr...",pJrM4yAZo0Gs63D9UCyCSQ
64864,/user_details?userid=tEZsTQEkZSK0RMjYZoZzZA,Hani A.,"Le Sud-Ouest, Canada",3,"rating for new terrace, too much noise coming ...",pJrM4yAZo0Gs63D9UCyCSQ
64865,/user_details?userid=z5xzL1Zl5iiHLuOqn07CLg,E T.,"Ottawa, Canada",5,Staff were friendly and helpful. The food was ...,pJrM4yAZo0Gs63D9UCyCSQ


In [4]:
def get_sentences(text, words):
    # take text and test every sentences for a list of words
    # if any word fromt the list in a sentence, join thoses sentenses into a list
    #return [sentence + '.' for sentence in x.split('.') if word in sentence]
    return [sentence + '.' for sentence in text.split('.') if any(w in sentence for w in words)]

In [5]:
import statsmodels.api as sm
import statistics
import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [149]:
food = ['food','taste','tasty','delicious','best','fresh','cold','hot','disgusting','dry']
service = ['service','staff','friendly','fast','quick','slow','wait','polite','rude']
value = ['value','price','portion','size','expensive','overpriced','affordable','reasonable']
experience = ['experience','ambiance','atmosphere','noise','noisy','mood','crowd']
clean = ['clean','dirty','filthy','gross','bathroom']

features = [food, service, value, experience, clean]

for f in features:

    words = f
    # get sentences containing a topic (feature)
    df[f[0]] = df['user_review'].apply(lambda x: get_sentences(x, words))
    # keep businesses with sentences
    review = df[['biz_id', 'user_rating', f[0]]].loc[df[f[0]].map(lambda d: len(d)) > 0]
    # apply sentiment analysis on sentences (calc the mean if more than one)
    review['sentiment'] = review[f[0]].apply(lambda l: statistics.mean([sid.polarity_scores(e)['compound'] for e in l]) )
    # average the sentiment by businesses
    rel = review.groupby(by='biz_id').mean().sort_values('sentiment', ascending=False)

    y = rel['user_rating']
    x = rel['sentiment']
    #y = review['user_rating']
    #x = review['sentiment']

    model = sm.OLS(y,sm.add_constant(x))
    results = model.fit(cov_type='HC2')
    #print(results.summary())
    print(f'{f[0]} - R-squared: {results.rsquared:.3f}, coef: {results.params["sentiment"]:.3f}, p-value : {results.pvalues["sentiment"]:.3f}')

food - R-squared: 0.240, coef: 1.710, p-value : 0.000
service - R-squared: 0.209, coef: 1.322, p-value : 0.000
value - R-squared: 0.138, coef: 1.158, p-value : 0.000
experience - R-squared: 0.268, coef: 1.354, p-value : 0.000
clean - R-squared: 0.321, coef: 1.524, p-value : 0.000


In [6]:
def get_best(df, food):
    df[food] = df['user_review'].apply(lambda x: get_sentences(x, [food]))
    review = df[['biz_id', 'user_rating', food]].loc[df[food].map(lambda d: len(d)) > 0]
    review['sentiment'] = review[food].apply(lambda l: statistics.mean([sid.polarity_scores(e)['compound'] for e in l]) )
    avg_sent = review.groupby(by='biz_id').mean().sort_values('sentiment', ascending=False)
    top = pd.merge(avg_sent, biz[['id', 'name', 'review_count', 'rating', 'price', 'location']], left_on='biz_id', right_on='id')
    return top

In [7]:
# search for any topic of keywords which businesses have the most positive sentiment for that topic
tmp = get_best(df, 'poutine')[0:10]
tmp

,user_rating,sentiment,id,name,review_count,rating,price,location
0,5.0,0.9872,_0KxeZoaK_ImMeyhbdG11A,Nevski,7,4.0,NaN,"['1228 Rue Stanley', 'Montreal, QC H3B 2S7', '..."
1,5.0,0.9782,8Utj3HIQ_Ma_uE8px-7nQg,Veuve Chalet,11,4.5,NaN,"['222 Avenue Laurier O', 'Montreal, QC H2T 2N8..."
2,4.0,0.9730,sKlZhq3Ag3lfLFNx9HsSUg,Amir,13,4.5,$$,"['1337 Avenue du Mont-Royal E', 'Montreal, QC ..."
3,5.0,0.9606,kfDtfA0dPm3rNl0ztSop1Q,L'Inconnu,16,4.0,NaN,"['5115 Boulevard Saint-Laurent', 'Montreal, QC..."
4,5.0,0.9569,ayCo510gQ9zDt-HijqYhQA,Bowhead Pub,18,4.0,NaN,"['3723 Boulevard Saint-Laurent', 'Montreal, QC..."
5,4.0,0.9236,LSPwq3J4WDHh6q3Cu0uMog,Régine Café,277,4.5,$$,"['1840 Beaubien E', 'Montreal, QC H1G 2L6', 'C..."
6,4.5,0.8891,Ok9vikKusbG_Ue9pJf6s3Q,La Capital Tacos,288,4.0,$$,"['1096 Boulevard Saint-Laurent', 'Montreal, QC..."
7,5.0,0.8885,wBXIdxs2XKqt9Q-r5-ik9w,Bulma Bar,13,4.5,NaN,"['686 Rue Notre-Dame O', 'Montreal, QC H3C 1J2..."
8,5.0,0.8881,JN8s_dgw9nrSzkHnXxNOtg,Dinette Triple Crown,254,4.5,$$,"['6704 Rue Clark', 'Montreal, QC H2S 3G2', 'Ca..."
9,4.0,0.8221,DLRpQJ6UtoQswcWpQGS7Sg,La Maison du Magret,47,4.5,$$$,"['102 Rue Saint-Antoine O', 'Montreal, QC H2Y ..."
